## **Figure 3**: Effect of Spatial Resolution on Noise in SWMMIN Simulations

#### Importing required libraries

In [1]:
import pyswmm                   # Library for working with SWMM (Storm Water Management Model)
import pandas as pd             # Data manipulation and analysis library
import numpy as np              
import pathlib                  # Object-oriented filesystem paths
import matplotlib as mpl        
import matplotlib.pyplot as plt  

# Set font properties for matplotlib
font = {'family' : 'Serif',
        'weight' : 'bold', 
        'size'   : 10}
mpl.rc('font', **font)  # Apply font settings to matplotlib
mpl.rc('font', **font)  # Apply font settings to matplotlib (duplicate line, can be removed)
mpl.rcParams['figure.dpi'] = 600  # Set the resolution of the figure
mpl.rcParams["mathtext.fontset"] = "dejavuserif"  # Set the font for math text

### Read input file names

In [2]:
# Directory for network inp files
dir = pathlib.Path('../../Networks/Linear Network')
# FIle names for network inp files
partial_name = 'Linear_Network_SWMMIN_'
filenames = ['1000m.inp','100m.inp','10m.inp','ASD.inp']

### Execute and Time each file

In [3]:
import time
execution_times = []
timing_iterations = 1 # Number of times to run the simulation for each network
stp=0       #steps counter
every=10000  #Interval of printing current time

# Running the simulation for each network
for filename in filenames:
    start_time = time.time()
    for iteration in range(timing_iterations):
        file_path = dir.joinpath(partial_name+ filename)
        print(file_path)
        sim = pyswmm.Simulation(inputfile=str(file_path), outputfile=str(file_path).replace('.inp','.out'))
        
        with sim as sim:
            for step in sim:
                if stp%every==0:
                    print('Current Time is >> ',sim.current_time)
                stp+=1
                pass
    execution_times.append(time.time()-start_time)

..\..\Networks\Linear Network\Linear_Network_SWMMIN_1000m.inp
Current Time is >>  2022-05-18 00:00:00
Current Time is >>  2022-05-18 02:46:40
Current Time is >>  2022-05-18 05:33:20
Current Time is >>  2022-05-18 08:20:00
Current Time is >>  2022-05-18 11:06:40
Current Time is >>  2022-05-18 13:53:20
Current Time is >>  2022-05-18 16:40:00
Current Time is >>  2022-05-18 19:26:40
Current Time is >>  2022-05-18 22:13:20
..\..\Networks\Linear Network\Linear_Network_SWMMIN_100m.inp
Current Time is >>  2022-05-18 01:01:00
Current Time is >>  2022-05-18 03:47:40
Current Time is >>  2022-05-18 06:34:20
Current Time is >>  2022-05-18 09:21:00
Current Time is >>  2022-05-18 12:07:40
Current Time is >>  2022-05-18 14:54:20
Current Time is >>  2022-05-18 17:41:00
Current Time is >>  2022-05-18 20:27:40
Current Time is >>  2022-05-18 23:14:20
..\..\Networks\Linear Network\Linear_Network_SWMMIN_10m.inp
Current Time is >>  2022-05-18 01:59:52
Current Time is >>  2022-05-18 04:46:32
Current Time is >

In [4]:
# Display average execution times
execution_times = [times/timing_iterations for times in execution_times]
print(execution_times)

[3.485758066177368, 3.8521854877471924, 22.675228357315063, 103.25178527832031]


## Extract Target Results from the output files

In [5]:
# Initialize empty DataFrames to store pressures and flowrates
pressures = pd.DataFrame()
flowrates = pd.DataFrame()

# Iterate over each filename in the list of filenames
for filename in filenames:
    last = -1  # Initialize the variable to track the last link or node
    file_path = dir.joinpath(partial_name + filename).with_suffix('.out')  # Construct the file path
    print(file_path)  # Print the file path for debugging purposes
    
    # Open the SWMM output file
    with pyswmm.Output(str(file_path)) as output:
        # Iterate over each link in the output
        for link in output.links:
            link_name = link.split('-')  # Split the link name by '-'
            if link_name[0] == 'P1':  # Check if the link starts with 'P1'
                if int(link_name[1]) > last:  # Update the last link if the current link number is greater
                    last = int(link_name[1])
        link_target = 'P1-' + str(last)  # Construct the target link name
        # Store the flow rate series in the flowrates DataFrame
        flowrates[filename.replace('.inp', '')] = output.link_series(link_target, 'Flow_rate').values()
        
        last = -1  # Reset the variable to track the last node
        # Iterate over each node in the output
        for node in output.nodes:
            node_name = node.split('-')  # Split the node name by '-'
            if node_name[0] == '1':  # Check if the node starts with '1'
                if int(node_name[1]) > last:  # Update the last node if the current node number is greater
                    last = int(node_name[1])
        if last == -1:
            node_target = 'DN1'  # Set the target node to 'DN1' if no nodes were found
        else:
            node_target = '1-' + str(last) + '-DN1'  # Construct the target node name
        # Store the depth series in the pressures DataFrame
        pressures[filename.replace('.inp', '')] = output.node_series(node_target, 'DEPTH').values()
        # Extract the time series from the depth series
        plot_time = list(output.node_series(node_target, 'DEPTH').keys())

..\..\Networks\Linear Network\Linear_Network_SWMMIN_1000m.out
..\..\Networks\Linear Network\Linear_Network_SWMMIN_100m.out
..\..\Networks\Linear Network\Linear_Network_SWMMIN_10m.out
..\..\Networks\Linear Network\Linear_Network_SWMMIN_ASD.out


In [6]:
pressures.max()

1000m    28.065231
100m     12.294351
10m       6.165298
ASD       6.133239
dtype: float64

In [7]:
plot_time = pd.Series(plot_time)
time_minutes = (plot_time - plot_time[0]).dt.total_seconds() / 60  # Convert to float hours
font = {'family' : 'Serif',
        'weight' : 'bold', 
        'size'   : 12}
mpl.rc('font', **font)

fig,ax=plt.subplots(2,4,sharex=True)
fig.set_figheight(11)
fig.set_figwidth(20)

ax[0,0].plot(time_minutes,flowrates['1000m'],label="1,000 m",c="#e66101")
ax[1,0].plot(time_minutes,pressures[('1000m')],c="#e66101")
ax[0,1].plot(time_minutes,flowrates["100m"],label="100 m",c="#fdb863")
ax[1,1].plot(time_minutes,pressures["100m"],c="#fdb863")
ax[0,2].plot(time_minutes,flowrates["10m"],label="10 m",c="#b2abd2")
ax[1,2].plot(time_minutes,pressures["10m"],c="#b2abd2")
ax[0,3].plot(time_minutes,flowrates["ASD"],label="Pachaly et al. (2020)'s ASD",c="#5e3c99")
ax[1,3].plot(time_minutes,pressures["ASD"],c="#5e3c99")
ax[0,0].annotate(r"$t_{exec}$ = "+str(round(execution_times[0],2))+ " s", xy=(30,280))
ax[0,1].annotate(r"$t_{exec}$ = "+str(round(execution_times[1],2))+ " s", xy=(30,280))
ax[0,2].annotate(r"$t_{exec}$ = "+str(round(execution_times[2],2))+ " s", xy=(28,280))
ax[0,3].annotate(r"$t_{exec}$ = "+str(round(execution_times[3],2))+ " s", xy=(27,280))
xlim1=(0,50)
for i in [0,1,2,3]:
    ax[1,i].set_ylim(0,15)
    ax[1,i].set_yticks(np.arange(0,15.1,3))
    ax[0,i].set_ylim(0,300)

ax[0, 0].set_title("Original (1,000 m)")
ax[0, 1].set_title("100 m")
ax[0, 2].set_title("10 m")
ax[0, 3].set_title("Pachaly et al. (2020)'s ASD")

ax[0,0].set_ylabel("Flow (LPS)")
ax[1,0].set_ylabel("Pressure (m)")

ax[1,0].annotate(str(round(pressures['1000m'].max(),1))+' m', xy=(10,12),xycoords = 'data', xytext = (15,12), arrowprops = dict(arrowstyle="->"))
# fig.legend(title = "$\Delta x$",loc="lower center",frameon=True,ncol=2, fontsize = 12, bbox_to_anchor = (0.5,-0.02,0,0))
plt.setp(ax, xlim=xlim1,xlabel="Time (min)")
plt.savefig("../Figure Files/Figure 3 - Noise Reduction",dpi=600)
plt.show

<function matplotlib.pyplot.show(close=None, block=None)>

### Optional: Test Inertial Damping Options

In [8]:
keep_filenames = [filename.replace('.inp', '_Keep.inp') for filename in filenames]
ignore_filenames = [filename.replace('.inp', '_Ignore.inp') for filename in filenames]

# Add columns in pressures and flowrates for the keep and ignore results
for filename in keep_filenames:
    pressures[filename.split('_')[-2] + "_Keep"] = np.nan
    flowrates[filename.split('_')[-2] + "_Keep"] = np.nan

for filename in ignore_filenames:
    pressures[filename.split('_')[-2] + "_Ignore"] = np.nan
    flowrates[filename.split('_')[-2] + "_Ignore"] = np.nan


pressures.head()

,1000m,100m,10m,ASD,1000m_Keep,100m_Keep,10m_Keep,ASD_Keep,1000m_Ignore,100m_Ignore,10m_Ignore,ASD_Ignore
0,0.000034,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000125,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000277,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000487,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000752,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
execution_times = []
timing_iterations = 5 # Number of times to run the simulation for each network
stp=0       #steps counter
every=10000  #Interval of printing current time

dir = dir/"Damping Test"

# Running the simulation for each network
for filename in keep_filenames + ignore_filenames:
    start_time = time.time()
    for iteration in range(timing_iterations):
        file_path = dir.joinpath(partial_name+ filename)
        print(file_path)
        sim = pyswmm.Simulation(inputfile=str(file_path), outputfile=str(file_path).replace('.inp','.out'))
        
        with sim as sim:
            for step in sim:
                if stp%every==0:
                    print('Current Time is >> ',sim.current_time)
                stp+=1
                pass
    execution_times.append(time.time()-start_time)

..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_1000m_Keep.inp
Current Time is >>  2022-05-18 00:00:00
Current Time is >>  2022-05-18 02:46:40
Current Time is >>  2022-05-18 05:33:20
Current Time is >>  2022-05-18 08:20:00
Current Time is >>  2022-05-18 11:06:40
Current Time is >>  2022-05-18 13:53:20
Current Time is >>  2022-05-18 16:40:00
Current Time is >>  2022-05-18 19:26:40
Current Time is >>  2022-05-18 22:13:20
..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_1000m_Keep.inp
Current Time is >>  2022-05-18 01:01:00
Current Time is >>  2022-05-18 03:47:40
Current Time is >>  2022-05-18 06:34:20
Current Time is >>  2022-05-18 09:21:00
Current Time is >>  2022-05-18 12:07:40
Current Time is >>  2022-05-18 14:54:20
Current Time is >>  2022-05-18 17:41:00
Current Time is >>  2022-05-18 20:27:40
Current Time is >>  2022-05-18 23:14:20
..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_1000m_Keep.inp
Current Time is >>  2022-05-18 02:02:00


In [10]:
# Iterate over each filename in the list of filenames
for filename in keep_filenames + ignore_filenames:
    last = -1  # Initialize the variable to track the last link or node
    file_path = dir.joinpath(partial_name + filename).with_suffix('.out')  # Construct the file path
    print(file_path)  # Print the file path for debugging purposes
    
    # Open the SWMM output file
    with pyswmm.Output(str(file_path)) as output:
        # Iterate over each link in the output
        for link in output.links:
            link_name = link.split('-')  # Split the link name by '-'
            if link_name[0] == 'P1':  # Check if the link starts with 'P1'
                if int(link_name[1]) > last:  # Update the last link if the current link number is greater
                    last = int(link_name[1])
        link_target = 'P1-' + str(last)  # Construct the target link name
        # Store the flow rate series in the flowrates DataFrame
        flowrates[filename.replace('.inp', '')] = output.link_series(link_target, 'Flow_rate').values()
        
        last = -1  # Reset the variable to track the last node
        # Iterate over each node in the output
        for node in output.nodes:
            node_name = node.split('-')  # Split the node name by '-'
            if node_name[0] == '1':  # Check if the node starts with '1'
                if int(node_name[1]) > last:  # Update the last node if the current node number is greater
                    last = int(node_name[1])
        if last == -1:
            node_target = 'DN1'  # Set the target node to 'DN1' if no nodes were found
        else:
            node_target = '1-' + str(last) + '-DN1'  # Construct the target node name
        # Store the depth series in the pressures DataFrame
        pressures[filename.replace('.inp', '')] = output.node_series(node_target, 'DEPTH').values()
        # Extract the time series from the depth series
        plot_time = list(output.node_series(node_target, 'DEPTH').keys())

..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_1000m_Keep.out
..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_100m_Keep.out
..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_10m_Keep.out
..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_ASD_Keep.out
..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_1000m_Ignore.out
..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_100m_Ignore.out
..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_10m_Ignore.out
..\..\Networks\Linear Network\Damping Test\Linear_Network_SWMMIN_ASD_Ignore.out


In [14]:
# Display average execution times
execution_times = [times/timing_iterations for times in execution_times]
print(execution_times)

[3.4266573905944826, 3.695289421081543, 22.47178373336792, 100.91991848945618, 3.6214082717895506, 3.591367244720459, 22.525697803497316, 98.57905893325805]


In [11]:
pressures

,1000m,100m,10m,ASD,1000m_Keep,100m_Keep,10m_Keep,ASD_Keep,1000m_Ignore,100m_Ignore,10m_Ignore,ASD_Ignore
0,0.000034,0.00000,0.000000,0.000000,0.000034,0.00000,0.000000,0.000000,0.000034,0.00000,0.000000,0.000000
1,0.000125,0.00000,0.000000,0.000000,0.000125,0.00000,0.000000,0.000000,0.000125,0.00000,0.000000,0.000000
2,0.000277,0.00000,0.000000,0.000000,0.000277,0.00000,0.000000,0.000000,0.000277,0.00000,0.000000,0.000000
3,0.000487,0.00000,0.000000,0.000000,0.000487,0.00000,0.000000,0.000000,0.000487,0.00000,0.000000,0.000000
4,0.000752,0.00000,0.000000,0.000000,0.000752,0.00000,0.000000,0.000000,0.000752,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
86335,0.028931,0.00003,0.015121,0.021289,0.028933,0.00003,0.015158,0.021059,0.028966,0.00003,0.015144,0.020995
86336,0.028931,0.00003,0.015121,0.021273,0.028932,0.00003,0.015159,0.021070,0.028966,0.00003,0.015146,0.021008
86337,0.028931,0.00003,0.015122,0.021254,0.028932,0.00003,0.015160,0.021078,0.028965,0.00003,0.015147,0.021021
86338,0.028931,0.00003,0.015122,0.021234,0.028932,0.00003,0.015161,0.021084,0.028965,0.00003,0.015148,0.021034


In [15]:
plot_time = pd.Series(plot_time)
time_minutes = (plot_time - plot_time[0]).dt.total_seconds() / 60  # Convert to float hours
font = {'family' : 'Serif',
        'weight' : 'bold', 
        'size'   : 12}
mpl.rc('font', **font)

fig,ax=plt.subplots(2,4,sharex=True)
fig.set_figheight(11)
fig.set_figwidth(20)

ax[0,0].plot(time_minutes,flowrates['1000m'],label="1,000 m",c="#e66101", linestyle = '--')
ax[0,0].plot(time_minutes,flowrates['1000m_Keep'],label="1,000 m Keep",c="#e66101")
ax[1,0].plot(time_minutes,pressures[('1000m')],c="#e66101", linestyle = '--')
ax[1,0].plot(time_minutes,pressures[('1000m_Keep')],c="#e66101")
ax[0,1].plot(time_minutes,flowrates["100m"],label="100 m",c="#fdb863", linestyle = '--')
ax[0,1].plot(time_minutes,flowrates["100m_Keep"],label="100 m Keep",c="#fdb863")
ax[1,1].plot(time_minutes,pressures["100m"],c="#fdb863", linestyle = '--')
ax[1,1].plot(time_minutes,pressures["100m_Keep"],c="#fdb863")
ax[0,2].plot(time_minutes,flowrates["10m"],label="10 m",c="#b2abd2", linestyle = '--')
ax[0,2].plot(time_minutes,flowrates["10m_Keep"],label="10 m Keep",c="#b2abd2")
ax[1,2].plot(time_minutes,pressures["10m"],c="#b2abd2", linestyle = '--')
ax[1,2].plot(time_minutes,pressures["10m_Keep"],c="#b2abd2")
ax[0,3].plot(time_minutes,flowrates["ASD"],label="Pachaly et al. (2020)'s ASD",c="#5e3c99", linestyle = '--')
ax[0,3].plot(time_minutes,flowrates["ASD_Keep"],label="Pachaly et al. (2020)'s ASD Keep",c="#5e3c99")
ax[1,3].plot(time_minutes,pressures["ASD"],c="#5e3c99", linestyle = '--')
ax[1,3].plot(time_minutes,pressures["ASD_Keep"],c="#5e3c99")

ax[0,0].annotate(r"$t_{exec}$ = "+str(round(execution_times[0],2))+ " s", xy=(30,280))
ax[0,1].annotate(r"$t_{exec}$ = "+str(round(execution_times[1],2))+ " s", xy=(30,280))
ax[0,2].annotate(r"$t_{exec}$ = "+str(round(execution_times[2],2))+ " s", xy=(28,280))
ax[0,3].annotate(r"$t_{exec}$ = "+str(round(execution_times[3],2))+ " s", xy=(27,280))

xlim1=(0,50)
for i in [0,1,2,3]:
    ax[1,i].set_ylim(0,15)
    ax[1,i].set_yticks(np.arange(0,15.1,3))
    ax[0,i].set_ylim(0,300)

ax[0, 0].set_title("Original (1,000 m)")
ax[0, 1].set_title("100 m")
ax[0, 2].set_title("10 m")
ax[0, 3].set_title("Pachaly et al. (2020)'s ASD")

ax[0,0].set_ylabel("Flow (LPS)")
ax[1,0].set_ylabel("Pressure (m)")

ax[1,0].annotate(str(round(pressures['1000m'].max(),1))+' m', xy=(10,12),xycoords = 'data', xytext = (15,12), arrowprops = dict(arrowstyle="->"))
# fig.legend(title = "$\Delta x$",loc="lower center",frameon=True,ncol=2, fontsize = 12, bbox_to_anchor = (0.5,-0.02,0,0))
plt.setp(ax, xlim=xlim1,xlabel="Time (min)")
plt.savefig("../Figure Files/Figure SXX - Noise Reduction - Keep Inertial Terms",dpi=600)
plt.show

<function matplotlib.pyplot.show(close=None, block=None)>

In [16]:
plot_time = pd.Series(plot_time)
time_minutes = (plot_time - plot_time[0]).dt.total_seconds() / 60  # Convert to float hours
font = {'family' : 'Serif',
        'weight' : 'bold', 
        'size'   : 12}
mpl.rc('font', **font)

fig,ax=plt.subplots(2,4,sharex=True)
fig.set_figheight(11)
fig.set_figwidth(20)

ax[0,0].plot(time_minutes,flowrates['1000m'],label="1,000 m",c="#e66101")
ax[0,0].plot(time_minutes,flowrates['1000m_Ignore'],label="1,000 m Ignore",c="#e66101",linestyle=':')
ax[1,0].plot(time_minutes,pressures[('1000m')],c="#e66101")
ax[1,0].plot(time_minutes,pressures[('1000m_Ignore')],c="#e66101",linestyle=':')
ax[0,1].plot(time_minutes,flowrates["100m"],label="100 m",c="#fdb863")
ax[0,1].plot(time_minutes,flowrates["100m_Ignore"],label="100 m Ignore",c="#fdb863",linestyle=':')
ax[1,1].plot(time_minutes,pressures["100m"],c="#fdb863")
ax[1,1].plot(time_minutes,pressures["100m_Ignore"],c="#fdb863",linestyle=':')
ax[0,2].plot(time_minutes,flowrates["10m"],label="10 m",c="#b2abd2")
ax[0,2].plot(time_minutes,flowrates["10m_Ignore"],label="10 m Ignore",c="#b2abd2",linestyle=':')
ax[1,2].plot(time_minutes,pressures["10m"],c="#b2abd2")
ax[1,2].plot(time_minutes,pressures["10m_Ignore"],c="#b2abd2",linestyle=':')
ax[0,3].plot(time_minutes,flowrates["ASD"],label="Pachaly et al. (2020)'s ASD",c="#5e3c99")
ax[0,3].plot(time_minutes,flowrates["ASD_Ignore"],label="Pachaly et al. (2020)'s ASD Ignore",c="#5e3c99",linestyle=':')
ax[1,3].plot(time_minutes,pressures["ASD"],c="#5e3c99")
ax[1,3].plot(time_minutes,pressures["ASD_Ignore"],c="#5e3c99",linestyle=':')

ax[0,0].annotate(r"$t_{exec}$ = "+str(round(execution_times[3],2))+ " s", xy=(30,280))
ax[0,1].annotate(r"$t_{exec}$ = "+str(round(execution_times[5],2))+ " s", xy=(30,280))
ax[0,2].annotate(r"$t_{exec}$ = "+str(round(execution_times[6],2))+ " s", xy=(28,280))
ax[0,3].annotate(r"$t_{exec}$ = "+str(round(execution_times[7],2))+ " s", xy=(27,280))
xlim1=(0,50)
for i in [0,1,2,3]:
    ax[1,i].set_ylim(0,16)
    ax[1,i].set_yticks(np.arange(0,15.1,3))
    ax[0,i].set_ylim(0,300)

ax[0, 0].set_title("Original (1,000 m)")
ax[0, 1].set_title("100 m")
ax[0, 2].set_title("10 m")
ax[0, 3].set_title("Pachaly et al. (2020)'s ASD")

ax[0,0].set_ylabel("Flow (LPS)")
ax[1,0].set_ylabel("Pressure (m)")

ax[1,0].annotate(str(round(pressures['1000m'].max(),1))+' m', xy=(10,12),xycoords = 'data', xytext = (15,12), arrowprops = dict(arrowstyle="->"))
# fig.legend(title = "$\Delta x$",loc="lower center",frameon=True,ncol=2, fontsize = 12, bbox_to_anchor = (0.5,-0.02,0,0))
plt.setp(ax, xlim=xlim1,xlabel="Time (min)")
plt.savefig("../Figure Files/Figure SXX - Noise Reduction - Ignore Inertial Terms",dpi=600)

plt.show

<function matplotlib.pyplot.show(close=None, block=None)>